In [ ]:
!pip install chromadb==0.5.5 langchain-chroma==0.1.2 langchain==0.2.11 langchain-community==0.2.10 langchain-text-splitters==0.2.2 langchain-groq==0.1.6 transformers==4.43.2 sentence-transformers==3.0.1 unstructured==0.15.0 unstructured[pdf]==0.15.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 18.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 4.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3

In [ ]:
import os

from langchain.document_loaders import UnstructuredFileLoader, OnlinePDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain_chroma import Chroma
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA

ModuleNotFoundError: No module named 'langchain'

In [ ]:
os.environ["GROQ_API_KEY"] = "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"

In [ ]:
# loading the document
loader = UnstructuredFileLoader("/content/NIPS-2017-attention-is-all-you-need-Paper.pdf")
documents = loader.load()

In [ ]:
# splittin into text chunks
text_splitter = CharacterTextSplitter(chunk_size=2000, chunk_overlap=400)
texts = text_splitter.split_documents(documents)

In [ ]:
texts = text_splitter.split_documents(documents)

In [ ]:
type(texts)

list

In [ ]:
len(texts)

20

In [ ]:
texts[:3]

[Document(metadata={'source': '/content/NIPS-2017-attention-is-all-you-need-Paper.pdf'}, page_content='Attention Is All You Need\n\nAshish Vaswani∗ Google Brain avaswani@google.com\n\nNoam Shazeer∗ Google Brain noam@google.com\n\nNiki Parmar∗ Google Research nikip@google.com\n\nJakob Uszkoreit∗ Google Research usz@google.com\n\nLlion Jones∗ Google Research llion@google.com\n\nAidan N. Gomez∗ † University of Toronto aidan@cs.toronto.edu\n\nŁukasz Kaiser∗ Google Brain lukaszkaiser@google.com\n\nIllia Polosukhin∗ ‡ illia.polosukhin@gmail.com\n\nAbstract\n\nThe dominant sequence transduction models are based on complex recurrent or convolutional neural networks that include an encoder and a decoder. The best performing models also connect the encoder and decoder through an attention mechanism. We propose a new simple network architecture, the Transformer, based solely on attention mechanisms, dispensing with recurrence and convolutions entirely. Experiments on two machine translation tasks

In [ ]:
embeddings = HuggingFaceEmbeddings()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
persist_directory = "/content/doc_db"

In [ ]:
vectordb = Chroma.from_documents( documents = texts,
                                 embedding = embeddings,
                                  persist_directory = persist_directory )

In [ ]:
# retriever
retriever = vectordb.as_retriever()

In [ ]:
# llm from Groq
llm = ChatGroq(
    model = "llama-3.1-70b-versatile",
    temperature=0
)

In [ ]:
# create a qa chain
qa_chain = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type = "stuff",
    retriever = retriever,
    return_source_documents = True
)

In [ ]:
# invoke the qa chain and get a response for user query
query = "what is the model architecture discussed in this paper?"
response = qa_chain.invoke({"query": query})

In [ ]:
print(response)

{'query': 'what is the model architecture discussed in this paper?', 'result': 'The model architecture discussed in this paper is called the Transformer. It consists of an encoder and a decoder, both of which are composed of a stack of identical layers.\n\nThe encoder has 6 identical layers, each with two sub-layers:\n\n1. A multi-head self-attention mechanism\n2. A position-wise fully connected feed-forward network\n\nThe decoder also has 6 identical layers, but with an additional third sub-layer that performs multi-head attention over the output of the encoder stack.\n\nThe model uses residual connections and layer normalization to facilitate the flow of information through the network. The output of each sub-layer is LayerNorm(x + Sublayer(x)), where Sublayer(x) is the function implemented by the sub-layer itself.\n\nThe model also uses multi-head attention, which allows the model to jointly attend to information from different representation subspaces at different positions. The mo

In [ ]:
print(response["result"])

The model architecture discussed in this paper is called the Transformer. It consists of an encoder and a decoder, both of which are composed of a stack of identical layers.

The encoder has 6 identical layers, each with two sub-layers:

1. A multi-head self-attention mechanism
2. A position-wise fully connected feed-forward network

The decoder also has 6 identical layers, but with an additional third sub-layer that performs multi-head attention over the output of the encoder stack.

The model uses residual connections and layer normalization to facilitate the flow of information through the network. The output of each sub-layer is LayerNorm(x + Sublayer(x)), where Sublayer(x) is the function implemented by the sub-layer itself.

The model also uses multi-head attention, which allows the model to jointly attend to information from different representation subspaces at different positions. The model uses 8 parallel attention layers, or heads, with a reduced dimensionality of 64 for eac

In [ ]:
print(response["source_documents"][0].metadata['source'])

/content/NIPS-2017-attention-is-all-you-need-Paper.pdf


In [ ]:
query = "what is the model architecture discussed in this paper?"
response = qa_chain.invoke({"query": query})
print(response["result"])
print("*"*30)
print("Source Document:", response["source_documents"][0].metadata["source"])

The model architecture discussed in this paper is called the Transformer. It consists of an encoder and a decoder, both of which are composed of a stack of identical layers. 

The encoder has 6 layers, each with two sub-layers: a multi-head self-attention mechanism and a position-wise fully connected feed-forward network. 

The decoder also has 6 layers, with three sub-layers: a multi-head self-attention mechanism, a multi-head attention mechanism over the output of the encoder stack, and a position-wise fully connected feed-forward network. 

The Transformer uses multi-head attention, which allows the model to jointly attend to information from different representation subspaces at different positions.
******************************
Source Document: /content/NIPS-2017-attention-is-all-you-need-Paper.pdf
